In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as plt
import stock_utils as su

from signaller import Signaller
from trading_algorithm import TradingAlgorithm

from sklearn.externals import joblib

%matplotlib inline

## Choose a ticker symbol and generate a dataset with selected new features for it

In [2]:
#create a dataset for AAPL
rs = 42 # random state
tickers = ['AAPL', 'XOM', 'JNJ']
symbol = tickers[0]
window = 5
look_ahead = 1

start_date = datetime.datetime(2017, 1, 1)
end_date = datetime.datetime(2017, 10, 30)

## Load saved models

In [3]:
import os
model_filenames = []
for file in os.listdir('models'):
    if file.endswith('.pkl'):
        model_filenames.append(file)

estimators = []
for filename in model_filenames:
    print('Loading model {0}'.format(filename))
    estimators.append(joblib.load('models/{0}'.format(filename)))

Loading model ElasticNet.pkl
Loading model LinearRegression.pkl
Loading model Lasso.pkl
Loading model Ridge.pkl


## Load stock data, calculate predictions and trade signals

In [4]:
#price variation in percentage which can inform trade decisions
delta = 0.01

signaller_params = {'delta' : delta}

stock_data = []

for symbol in tickers:
    print('Processing data for {0}'.format(symbol))
    dataset = su.preprocess_data(symbol, window, look_ahead, start_date, end_date)

    #calculate the predicted prices
    print('Calculating predictions for {0}'.format(symbol))
    pred = su.calculate_predictions_for_dataset(dataset=dataset, estimators=estimators, plot_and_print=False)
    
    #add the predicted prices to the dataset
    dataset['Predicted Price'] = pred
    
    #calculate trade signals based on different indicators and add those signals to the dataframe
    print('Calculating signals for {0}\n'.format(symbol))
    signaller = Signaller(signaller_params)

    dataset = signaller.calculate_trade_signal(dataset)
    
    stock_data.append({'name' : symbol, 'data' : dataset})

Processing data for AAPL
Calculating predictions for AAPL
Calculating signals for AAPL

Processing data for XOM
Calculating predictions for XOM
Calculating signals for XOM

Processing data for JNJ
Calculating predictions for JNJ
Calculating signals for JNJ



## Run the trading algorithm and check the results

In [5]:
trading_algo = TradingAlgorithm()
profits = trading_algo.trade(stock_data)

for p in profits:
    symbol = p['name']
    profits_df = p['profits']
    
    print('Profits for {0}'.format(symbol))
#     print(profits_df[:6])
    
    print('Normal traders mean profit: {0:.2f}%'.format(profits_df['Normal Traders'].mean()))
    print('Prediction traders mean profit: {0:.2f}%\n'.format(profits_df['Prediction Traders'].mean()))

Trading stock AAPL
Balanced Trader no predictions starts trading
After 0 transactions, Balanced Trader no predictions has 500 stocks and 58168.59$ cash spent and 0.00$ cash made
Balanced Trader no predictions ends trading with a profit of 0.00%

Balanced Trader with predictions starts trading
After 0 transactions, Balanced Trader with predictions has 500 stocks and 58168.59$ cash spent and 0.00$ cash made
Balanced Trader with predictions ends trading with a profit of 0.00%

Bearish Trader no predictions starts trading
Bearish Trader no predictions sells 100 stocks for 11633.72 $ on 2017-01-06 00:00:00
Bearish Trader no predictions sells 100 stocks for 12025.42 $ on 2017-01-25 00:00:00
Bearish Trader no predictions sells 100 stocks for 12703.26 $ on 2017-02-01 00:00:00
Bearish Trader no predictions sells 100 stocks for 12681.55 $ on 2017-02-02 00:00:00
Bearish Trader no predictions sells 100 stocks for 12735.82 $ on 2017-02-03 00:00:00
After 5 transactions, Bearish Trader no predictions

In [6]:
profits[0]['profits'].head()

,Normal Traders,Prediction Traders
0,0.000000,0.000000
1,6.208125,9.047309
2,22.166360,25.756111


In [7]:
profits[1]['profits'].head()

,Normal Traders,Prediction Traders
0,0.000000,0.000000
1,-4.517840,-3.812498
2,1.592323,0.886405


In [8]:
profits[2]['profits'].head()

,Normal Traders,Prediction Traders
0,0.000000,0.000000
1,1.647463,-0.995701
2,19.360657,17.116753
